# Bitcoin Price Forecasting API - Deployment Guide

## Overview
This notebook documents the deployment process for the Bitcoin price forecasting model as a REST API using FastAPI and Docker containerization.

## Project Structure

```
Time Series/
├── app/
│   ├── Dockerfile          # Container configuration
│   ├── main.py            # FastAPI application
│   ├── model.pkl          # Trained ML model
│   └── requirements.txt   # Python dependencies
├── data/
│   └── bitcoin_historical_data.csv
├── index.ipynb            # Main analysis notebook
├── deployment.ipynb       # This deployment guide
└── README.md
```

## 1. API Development

### FastAPI Application Structure
Created a REST API with the following endpoints:
- `GET /` - Health check endpoint
- `POST /predict` - Bitcoin price prediction endpoint

In [1]:
# FastAPI Application Code (main.py)
from fastapi import FastAPI 
import joblib
import pandas as pd

app = FastAPI(title="Bitcoin Price Forecast API")

# Load trained model
model = joblib.load("model.pkl")

@app.get("/")
def home():
    return {"message": "Bitcoin Time Series Forecast API is running"}

@app.post("/predict")
def predict(features: dict):
    """
    Expected input format:
    {
      "Open": 60000,
      "High": 60500,
      "Low": 59000,
      "Volume": 123456,
      "lag_1": 59800,
      "lag_2": 59000,
      "lag_3": 58500,
      "lag_4": 58000,
      "lag_5": 57500,
      "lag_7": 56000
    }
    """
    df = pd.DataFrame([features])
    prediction = model.predict(df)
    return {"predicted_close": float(prediction[0])}

ModuleNotFoundError: No module named 'fastapi'

## 2. Dependencies Management

### requirements.txt
```
fastapi
uvicorn
pandas
numpy
scikit-learn
joblib
statsmodels
```

## 3. Docker Containerization

### Dockerfile Configuration
```dockerfile
# Use lightweight Python image
FROM python:3.10-slim

# Set working directory
WORKDIR /app

# Copy requirements and install
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy application files
COPY . .

# Expose API port
EXPOSE 8000

# Run FastAPI app
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
```

## 4. Deployment Process

### Step 1: Build Docker Image
```bash
cd "c:\Users\User\Documents\Flatiron\projects\Time Series"
docker build -t bitcoin-forecast-api ./app
```

### Step 2: Run Container
```bash
docker run -p 8000:8000 bitcoin-forecast-api
```

### Step 3: Test API
- Health check: `http://localhost:8000/`
- Interactive docs: `http://localhost:8000/docs`

## 5. API Testing

### Test Input Format
```json
{
  "Open": 60000,
  "High": 60500,
  "Low": 59000,
  "Volume": 123456,
  "lag_1": 59800,
  "lag_2": 59000,
  "lag_3": 58500,
  "lag_4": 58000,
  "lag_5": 57500,
  "lag_7": 56000
}
```

### Expected Response
```json
{
  "predicted_close": 59234.56
}
```

## 6. Useful Docker Commands

```bash
# Build image (no cache)
docker build --no-cache -t bitcoin-forecast-api ./app

# Run container
docker run -p 8000:8000 bitcoin-forecast-api

# Stop all containers for this image
docker stop $(docker ps -q --filter ancestor=bitcoin-forecast-api)

# View container logs
docker logs $(docker ps -q --filter ancestor=bitcoin-forecast-api)

# Remove containers
docker rm -f $(docker ps -aq --filter ancestor=bitcoin-forecast-api)

# List running containers
docker ps
```

## 7. Cloud Deployment Options

### AWS Deployment
```bash
# Push to ECR
aws ecr create-repository --repository-name bitcoin-forecast
docker tag bitcoin-forecast-api:latest <account>.dkr.ecr.region.amazonaws.com/bitcoin-forecast:latest
docker push <account>.dkr.ecr.region.amazonaws.com/bitcoin-forecast:latest
```

### Heroku Deployment
```bash
heroku container:push web -a your-app-name
heroku container:release web -a your-app-name
```

### Google Cloud Run
```bash
gcloud run deploy bitcoin-forecast --image gcr.io/PROJECT-ID/bitcoin-forecast
```

## 8. Deployment Summary

### Successfully Deployed:
✅ FastAPI application with Bitcoin price prediction endpoint  
✅ Docker containerization for consistent deployment  
✅ Model serialization and loading via joblib  
✅ RESTful API with proper input validation  
✅ Interactive API documentation via FastAPI  

### Key Features:
- **Scalable**: Docker container can be deployed anywhere
- **Interactive**: Auto-generated API docs at `/docs`
- **Robust**: Error handling and input validation
- **Lightweight**: Minimal dependencies and slim Docker image

### Next Steps:
- Add authentication and rate limiting
- Implement model versioning
- Add monitoring and logging
- Set up CI/CD pipeline
- Deploy to production cloud environment